# 드라이브연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# 데이터 가져오기

In [ ]:
data_path = '/content/drive/MyDrive/빅프로젝트/코드/추천시스템'

tfidf_data = pd.read_csv(data_path+'/data_latlong.csv', encoding='cp949')
tfidf_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4388 entries, 0 to 4387
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   num                    4388 non-null   int64  
 1   category               4388 non-null   int64  
 2   T                      4388 non-null   int64  
 3   car_type               4388 non-null   int64  
 4   is_freezer             4388 non-null   int64  
 5   vibration-free         4388 non-null   int64  
 6   transport_type         4388 non-null   int64  
 7   Departure (1)          4388 non-null   object 
 8   Departure (2)          4388 non-null   object 
 9   Departure (3)          4388 non-null   object 
 10  Destination (1)        4388 non-null   object 
 11  Destination (2)        4388 non-null   object 
 12  Destination (3)        4388 non-null   object 
 13  Destination type       4388 non-null   float64
 14  Departure_latitude     3283 non-null   float64
 15  Depa

In [ ]:
tfidf_data.head()

,num,category,T,car_type,is_freezer,vibration-free,transport_type,Departure (1),Departure (2),Departure (3),Destination (1),Destination (2),Destination (3),Destination type,Departure_latitude,Departure_longitude,Destination_latitude,Destination_longitude
0,46640,3,3,1,0,2,3,충청남도,당진군,송악읍,대전광역시,중구,안영동,7.0,36.904159,126.699458,36.281500,127.377800
1,46669,3,3,1,0,2,2,경기도,화성시,팔탄면,경기도,화성시,동탄면,7.0,37.162248,126.904961,NaN,NaN
2,46673,3,6,1,0,2,3,강원도,춘천시,동면,부산광역시,사상구,모라동,7.0,37.910276,127.781058,35.186980,128.989962
3,46676,5,3,1,0,2,3,대전광역시,서구,관저동,인천광역시,중구,북성동,6.0,36.303951,127.337361,37.475161,126.618491
4,46683,1,3,4,1,2,3,충청북도,진천군,광혜원면,경기도,안성시,석정동,6.0,36.993408,127.435584,37.009900,127.259400


In [ ]:
tfidf_data.dropna(axis=1, inplace=True)
tfidf_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4388 entries, 0 to 4387
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   num               4388 non-null   int64  
 1   category          4388 non-null   int64  
 2   T                 4388 non-null   int64  
 3   car_type          4388 non-null   int64  
 4   is_freezer        4388 non-null   int64  
 5   vibration-free    4388 non-null   int64  
 6   transport_type    4388 non-null   int64  
 7   Departure (1)     4388 non-null   object 
 8   Departure (2)     4388 non-null   object 
 9   Departure (3)     4388 non-null   object 
 10  Destination (1)   4388 non-null   object 
 11  Destination (2)   4388 non-null   object 
 12  Destination (3)   4388 non-null   object 
 13  Destination type  4388 non-null   float64
dtypes: float64(1), int64(7), object(6)
memory usage: 480.1+ KB


# 데이터 전처리

In [ ]:
tfidf_data = tfidf_data.drop(tfidf_data.loc[tfidf_data['Departure (1)'].str.contains('제주')].index)
tfidf_data = tfidf_data.drop(tfidf_data.loc[tfidf_data['Destination (1)'].str.contains('제주')].index)
tfidf_data = tfidf_data.reset_index(drop=True)
tfidf_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4257 entries, 0 to 4256
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   num               4257 non-null   int64  
 1   category          4257 non-null   int64  
 2   T                 4257 non-null   int64  
 3   car_type          4257 non-null   int64  
 4   is_freezer        4257 non-null   int64  
 5   vibration-free    4257 non-null   int64  
 6   transport_type    4257 non-null   int64  
 7   Departure (1)     4257 non-null   object 
 8   Departure (2)     4257 non-null   object 
 9   Departure (3)     4257 non-null   object 
 10  Destination (1)   4257 non-null   object 
 11  Destination (2)   4257 non-null   object 
 12  Destination (3)   4257 non-null   object 
 13  Destination type  4257 non-null   float64
dtypes: float64(1), int64(7), object(6)
memory usage: 465.7+ KB


# 시도별로 빈도가 많은 지역 확인
- 빈도가 적다면 차주가 물류를 하차하고 나서 근처에서 상차할 가능성이 낮아짐으로 공차의 확률이 높아짐을 의미

In [ ]:
tfidf_data['Departure (1)'].value_counts()

경기도        749
경상남도       444
부산광역시      438
전라남도       330
강원도        279
인천광역시      271
충청남도       266
전라북도       219
충청북도       217
서울특별시      211
경상북도       196
광주광역시      184
대전광역시      170
대구광역시      149
울산광역시      116
세종특별자치시     18
Name: Departure (1), dtype: int64

In [ ]:
cnt_df = pd.DataFrame()
cnt_df['location'] = list(tfidf_data['Departure (1)'].value_counts().index)
cnt_df['cnt'] = list(tfidf_data['Departure (1)'].value_counts().values)
cnt_df['percentage'] = round(cnt_df['cnt']/sum(cnt_df['cnt']), 3)

In [ ]:
cnt_df

,location,cnt,percentage
0,경기도,749,0.176
1,경상남도,444,0.104
2,부산광역시,438,0.103
3,전라남도,330,0.078
4,강원도,279,0.066
5,인천광역시,271,0.064
6,충청남도,266,0.062
7,전라북도,219,0.051
8,충청북도,217,0.051
9,서울특별시,211,0.050


# 가격에 비율 넣기

In [ ]:
list(cnt_df.loc[cnt_df['percentage'] >= 0.1, 'location'])

['경기도', '경상남도', '부산광역시']

In [ ]:
list(cnt_df.loc[(cnt_df['percentage'] < 0.1) & (cnt_df['percentage'] >= 0.05), 'location'])

['전라남도', '강원도', '인천광역시', '충청남도', '전라북도', '충청북도', '서울특별시']

In [ ]:
list(cnt_df.loc[cnt_df['percentage'] < 0.05, 'location'])

['경상북도', '광주광역시', '대전광역시', '대구광역시', '울산광역시', '세종특별자치시']

In [ ]:
# 입력데이터: 출발데이터
start_add = '광주광역시 동래구 명륜동 347'
real_final_fee = 800000

location = start_add.split()[0]

minus_location = ['경기도', '경상남도', '부산광역시']
still_location = ['전라남도', '강원도', '인천광역시', '충청남도', '전라북도', '충청북도', '서울특별시']
plus_location = ['경상북도', '광주광역시', '대전광역시', '대구광역시', '울산광역시', '세종특별자치시']

if location in minus_location:
    location_fee = -(real_final_fee * 0.05)
elif location in still_location:
    location_fee = 0
elif location in plus_location:
    location_fee = real_final_fee * 0.05

real_final_fee = real_final_fee + location_fee

In [ ]:
location_fee

40000.0